# Operacje na TensorFlow TensorachTa sekcja obejmuje:

* Indeksowanie i wycinanie* Zmianę kształtu z `tf.reshape`
* Operacje arytmetyczne i statystyczne
* Mnożenie macierzy oraz iloczyny skalarne
* Różnice względem PyTorch (np. brak operacji in-place)

In [1]:
import tensorflow as tf

## Indeksowanie i wycinanie

Tensory TensorFlow obsługują indeksowanie podobne do NumPy.

Warto jednak pamiętać, że zwracane obiekty to wciąż tensory.

<img src='../_img/arrayslicing.png' width="500" style="display: inline-block"><br><br>

In [5]:
row0 = tf.range(6)
matrix = x = row0 + tf.expand_dims(row0, 1) * 10

# Lub tf.newaxis
x = row0 + row0[:, tf.newaxis] * 10

print(x[0, 3:5])
print(x[4:, 4:])
print(x[:,2])
print(x[2::2,::2])


tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor(
[[44 45]
 [54 55]], shape=(2, 2), dtype=int32)
tf.Tensor([ 2 12 22 32 42 52], shape=(6,), dtype=int32)
tf.Tensor(
[[20 22 24]
 [40 42 44]], shape=(2, 3), dtype=int32)


## Zmiana kształtu

W TensorFlow korzystamy z funkcji `tf.reshape`. W przeciwieństwie do PyTorch nie ma metody `.view()`, ale działanie jest analogiczne.

In [7]:
reshaped = tf.reshape(matrix, (3, 12))
print(reshaped)

tf.Tensor(
[[ 0  1  2  3  4  5 10 11 12 13 14 15]
 [20 21 22 23 24 25 30 31 32 33 34 35]
 [40 41 42 43 44 45 50 51 52 53 54 55]], shape=(3, 12), dtype=int32)


## Arytmetyka tensorów

Operacje można zapisywać symbolicznie (`a + b`) lub poprzez funkcje z modułu `tf.math`.

TensorFlow nie wspiera operacji in-place na tensorach – **to istotna różnica względem PyTorch.**

In [8]:
a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
b = tf.constant([[10., 20., 30.], [40., 50., 60.]])
print('Dodawanie:', a + b)
print('Mnożenie elementowe:', tf.multiply(a, b))
print('Potęgowanie:', tf.pow(a, 2))

Dodawanie: tf.Tensor(
[[11. 22. 33.]
 [44. 55. 66.]], shape=(2, 3), dtype=float32)
Mnożenie elementowe: tf.Tensor(
[[ 10.  40.  90.]
 [160. 250. 360.]], shape=(2, 3), dtype=float32)
Potęgowanie: tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)


### Broadcasting

Reguły broadcastingu są takie jak w NumPy i PyTorch – dopasowujemy rozmiary od końca kształtu.

In [9]:
row = tf.constant([[1., 2., 3.]])

print(a + row)

tf.Tensor(
[[2. 4. 6.]
 [5. 7. 9.]], shape=(2, 3), dtype=float32)


### Iloczyn skalarny i mnożenie macierzy

Najczęściej korzystamy z `tf.tensordot` i `tf.matmul`.

Dla wektorów `tf.tensordot` działa jak `torch.dot`.

`tf.matmul`

Mnożenie macierzy (2D).

Działa na ostatnich 2 wymiarach

In [29]:
A = tf.reshape(tf.range(4), (2, 2))
B = tf.reshape(tf.range(4), (2, 2))
C = tf.matmul(A, B)  # shape: (2, 2)
print(C.shape)
print(A, B, C, sep='\n')


(2, 2)
tf.Tensor(
[[0 1]
 [2 3]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[0 1]
 [2 3]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  3]
 [ 6 11]], shape=(2, 2), dtype=int32)


In [34]:
A = tf.reshape(tf.range(27), (3, 3, 3))
B = tf.reshape(tf.range(27), (3, 3, 3))
print(A)

tf.Tensor(
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]], shape=(3, 3, 3), dtype=int32)


In [35]:
print(B)

tf.Tensor(
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]], shape=(3, 3, 3), dtype=int32)


Dla tensorów kształtu `(batch, rows, cols)`:
- `A.shape = (3, 3, 3)` - 3 macierze 3×3
- `B.shape = (3, 3, 3)` - 3 macierze 3×3
- `C.shape = (3, 3, 3)` - 3 wynikowe macierze 3×3

`matmul` mnoży odpowiednie pary macierzy:
- `A[0]` × `B[0]` → `C[0]`
- `A[1]` × `B[1]` → `C[1]`
- `A[2]` × `B[2]` → `C[2]`

Każde mnożenie to standardowe mnożenie macierzy 3×3:

```python
# Przykład dla pierwszej pary:
# A[0] = [[0,1,2], [3,4,5], [6,7,8]]
# B[0] = [[0,1,2], [3,4,5], [6,7,8]]
# C[0] = A[0] @ B[0] = macierz 3×3
```


In [37]:

C = tf.matmul(A, B)
print(C)


tf.Tensor(
[[[  15   18   21]
  [  42   54   66]
  [  69   90  111]]

 [[ 366  396  426]
  [ 474  513  552]
  [ 582  630  678]]

 [[1203 1260 1317]
  [1392 1458 1524]
  [1581 1656 1731]]], shape=(3, 3, 3), dtype=int32)


In [26]:
A = tf.eye(2, 2) + 1
B = tf.eye(2,2)
C = tf.matmul(A, B)  # shape: (3, 3)
print(C.shape)
C

(2, 2)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 1.],
       [1., 2.]], dtype=float32)>

In [15]:
A = tf.random.normal((3, 4))
B = tf.random.normal((4, 5))
C = tf.tensordot(A, B, axes=0)  # shape: (3, 5)
print(C.shape)
C

(3, 4, 4, 5)


<tf.Tensor: shape=(3, 4, 4, 5), dtype=float32, numpy=
array([[[[-7.21118040e-03, -7.06384555e-02, -1.34808972e-01,
           2.03060955e-02, -1.33329228e-01],
         [-8.54519010e-02,  2.12327801e-02,  2.79083662e-02,
           5.73648773e-02, -1.17181288e-02],
         [ 1.00872964e-02, -1.98004209e-02,  3.99751104e-02,
           8.55055973e-02,  8.35616142e-02],
         [ 4.15276736e-02, -1.02254018e-01, -1.47219785e-02,
           1.87631041e-01, -2.65930197e-03]],

        [[ 1.05472036e-01,  1.03317082e+00,  1.97174060e+00,
          -2.97000617e-01,  1.95009756e+00],
         [ 1.24983501e+00, -3.10554504e-01, -4.08192843e-01,
          -8.39029074e-01,  1.71391472e-01],
         [-1.47538632e-01,  2.89604545e-01, -5.84683239e-01,
          -1.25062037e+00, -1.22218728e+00],
         [-6.07391298e-01,  1.49558580e+00,  2.15326324e-01,
          -2.74432564e+00,  3.88954319e-02]],

        [[-1.35377198e-01, -1.32611239e+00, -2.53080082e+00,
           3.81211132e-01, -2.503

In [ ]:
vec1 = tf.constant([1., 2., 3.])
vec2 = tf.constant([4., 5., 6.])

dot = tf.tensordot(vec1, vec2, axes=1)

print('Iloczyn skalarny:', dot.numpy())

mat1 = tf.constant([[1., 2.], [3., 4.]])
mat2 = tf.constant([[5., 6.], [7., 8.]])
print('Mnożenie macierzy:', tf.matmul(mat1, mat2).numpy())

### Statystyki opisowe

W TensorFlow odpowiadają za to funkcje `tf.reduce_*`.

In [38]:
print('Suma:', tf.reduce_sum(a).numpy())

print('Średnia:', tf.reduce_mean(a).numpy())

print('Maksimum:', tf.reduce_max(a).numpy())

print('Minimum:', tf.reduce_min(a).numpy())

Suma: 21.0
Średnia: 3.5
Maksimum: 6.0
Minimum: 1.0


### Dodatkowe operacje

* `tf.linalg.norm(a)` – norma euklidesowa
* `tf.math.reduce_std(a)` – odchylenie standardowe
* `tf.reshape`, `tf.transpose`, `tf.concat` – operacje kształtuTensorFlow nie oferuje operacji in-place. Zamiast `tensor.add_()` tworzymy nowy tensor albo korzystamy z `tf.Variable`.

### Świetna robota!